# Fetching Telco Customer Churn Dataset from Kaggle

In [ ]:
import warnings
import joblib
from datetime import datetime
from pathlib import Path
import subprocess
import os
import zipfile

import pandas as pd

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Various settings
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.precision", 4)
pd.set_option("display.max_columns", None)

## Experimenting with Python Tools

In [ ]:
# Let's define useful paths

In [ ]:
from churn_detection.paths import EXTERNAL_DATA_DIR

In [ ]:
CURRENT_DIR = Path().cwd()

In [ ]:
zip_file = Path("telco-customer-churn.zip")
csv_file = Path("WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
# Let's download and unzip the dataset using Kaggle package

In [ ]:
subprocess.run(['kaggle', 'datasets', 'download', '-d', 'blastchar/telco-customer-churn'])

In [ ]:
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(CURRENT_DIR)

In [ ]:
# Let's read the extracted CSV file
data = pd.read_csv(csv_file)  
data.head()

In [ ]:
# Let's convert it to feather format into data directory
data.to_feather(EXTERNAL_DATA_DIR / "customer_churn.feather")

In [ ]:
# Let's remove the downloaded files
subprocess.run(['cmd', '/c', 'del', str(zip_file)], check=True)

In [ ]:
subprocess.run(['cmd', '/c', 'del', str(csv_file)], check=True)

## Refactoring as Modular Code

In [ ]:
from churn_detection import data

In [ ]:
target_dataset = "blastchar/telco-customer-churn"

In [ ]:
churn_data = data.fetch_batch_data(target_dataset)

In [ ]:
churn_data.head()

In [ ]:
data.save_batch_data(churn_data)